In [132]:
import pickle
from web3 import Web3

with open("spam_model.pkl", "rb") as f:
    model, vectorizer = pickle.load(f)

In [154]:
INFURA_URL = "https://sepolia.infura.io/v3/30ad3a534dc24d24a2c4f1af2a8f9b58"
w3 = Web3(Web3.HTTPProvider(INFURA_URL))

contract_address = "0x694aD0c48B86Adbd83C52d9224E0F45Cd74C5CDE"
contract_abi = [
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "message",
				"type": "string"
			},
			{
				"internalType": "bool",
				"name": "isSpam",
				"type": "bool"
			}
		],
		"name": "markAsSpam",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "string",
				"name": "message",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "bool",
				"name": "isSpam",
				"type": "bool"
			},
			{
				"indexed": False,
				"internalType": "bytes32",
				"name": "messageHash",
				"type": "bytes32"
			}
		],
		"name": "SpamReported",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "message",
				"type": "string"
			}
		],
		"name": "checkIfSpam",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "message",
				"type": "string"
			}
		],
		"name": "getMessageStatus",
		"outputs": [
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "bytes32",
				"name": "",
				"type": "bytes32"
			}
		],
		"name": "spamReports",
		"outputs": [
			{
				"internalType": "string",
				"name": "message",
				"type": "string"
			},
			{
				"internalType": "bool",
				"name": "isSpam",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]

contract = w3.eth.contract(address=contract_address, abi=contract_abi)

In [156]:
def predict_spam(message):
    X_input = vectorizer.transform([message])
    prediction = model.predict(X_input)[0]
    return bool(prediction)

In [164]:
def report_spam_to_blockchain(message, sender_private_key):
    is_spam = predict_spam(message)
    sender_account = w3.eth.account.from_key(sender_private_key)
    nonce = w3.eth.get_transaction_count(sender_account.address)

    tx = contract.functions.markAsSpam(message, is_spam).build_transaction({
        'from': sender_account.address,
        'nonce': nonce,
        'gas': 200000,
        'gasPrice': w3.to_wei('10', 'gwei')
    })

    signed_tx = w3.eth.account.sign_transaction(tx, sender_private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)


    print(f"Transaction sent! Hash: {w3.to_hex(tx_hash)}")
    return w3.to_hex(tx_hash)

In [166]:
def check_spam_status_on_blockchain(message):
    return contract.functions.getMessageStatus(message).call()

In [168]:
sender_private_key = "0x7102fe96860235d64422d97ac2a72a89b9632c21a876605bb39c1fa1b027a3fa"  

message = "Hello Sabareesh, How are you doing?"
message = "Congratulations! You've won a free iPhone!"

tx_hash = report_spam_to_blockchain(message, sender_private_key)
print(f"Spam report transaction hash: {tx_hash}")

status = check_spam_status_on_blockchain(message)
print(f"Blockchain reports: {status}")

print(f"Check transaction details: https://sepolia.etherscan.io/tx/{tx_hash}")

Transaction sent! Hash: 0x6b5a481f2c3a58253022a383633297b035027b47694a182fd7e9b10a76ff004a
Spam report transaction hash: 0x6b5a481f2c3a58253022a383633297b035027b47694a182fd7e9b10a76ff004a
Blockchain reports: Not Spam
Check transaction details: https://sepolia.etherscan.io/tx/0x6b5a481f2c3a58253022a383633297b035027b47694a182fd7e9b10a76ff004a
